<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/dataset_modification/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

setup to import important files from google drive or from github 


In [1]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/Incremental-Learning.git # clone proj repository
!rm -rf Incremental-Learning/README.md 
!rm -rf Incremental-Learning/baselines.ipynb
sys.path.append('Incremental-Learning/')

fatal: destination path 'Incremental-Learning' already exists and is not an empty directory.


import libraries


In [2]:
import torch
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
import numpy as np

# project classes --------------------------------------------------------------
from dataset.ilcifar100 import ilCIFAR100
from baselines.resnet import resnet32

In [3]:
# Define params

NUM_RUN = 3
SEEDS = [1,2,3]
NUM_EPOCHS = 70
BATCH_SIZE = 128
NUM_CLASS_BATCH = 10

In [5]:
# Define transformations for training
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define transformations for evaluation
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

FINE TUNING

In [6]:
# dataloaders dict initialization
dataloaders = {run:{batch:{'train':None, 'val':None, 'test':None} for batch in range(NUM_CLASS_BATCH)} for run in range(NUM_RUN)}

# make dataloaders
for run in range(NUM_RUN):
    
    original_training_set = ilCIFAR100(root="cifar", seed=SEEDS[run], transform=train_transform)
    original_test_set = ilCIFAR100(root="cifar", seed=SEEDS[run], transform=test_transform)
    
    for class_batch in range(NUM_CLASS_BATCH):
        
        # get indices
        train_indices = original_training_set.batches[class_batch]['train']
        val_indices = original_training_set.batches[class_batch]['val']
        test_indices = original_test_set.batches[class_batch]

        # make subsets
        train_set = Subset(dataset=original_training_set, indices=train_indices)
        val_set = Subset(dataset=original_training_set, indices=val_indices)
        test_set = Subset(dataset=original_test_set, indices=test_indices)

        # make dataloaders
        train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
        val_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
        test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

        # update dataloaders dict
        dataloaders[run][class_batch]['train'] = train_dataloader
        dataloaders[run][class_batch]['val'] = val_dataloader
        dataloaders[run][class_batch]['test'] = test_dataloader

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
for inputs, labels in dataloaders[0][1]['train']:
    print(inputs)
   

Streaming output truncated to the last 5000 lines.

         [[-0.5843, -0.6000, -0.6314,  ..., -1.0000, -1.0000, -1.0000],
          [-0.6706, -0.6314, -0.5059,  ..., -1.0000, -1.0000, -1.0000],
          [-0.4980, -0.5137, -0.4431,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

         [[-0.6078, -0.6235, -0.6549,  ..., -1.0000, -1.0000, -1.0000],
          [-0.6784, -0.6392, -0.5059,  ..., -1.0000, -1.0000, -1.0000],
          [-0.5294, -0.5373, -0.4667,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]]])
tensor([[[[ 0.2549,  0.2627,  0.2863,  ..., -0.70